# Stoneburner, Kurt
- ## DSC 530 - Week 11

In [1]:
def CleanData(resp):
    """Cleans respondent data.

    resp: DataFrame
    """
    resp.cmdivorcx.replace([9998, 9999], np.nan, inplace=True)

    resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)
    resp['duration'] = (resp.cmdivorcx - resp.cmmarrhx) / 12.0
    resp['durationsofar'] = (resp.cmintvw - resp.cmmarrhx) / 12.0

    month0 = pd.to_datetime('1899-12-15')
    dates = [month0 + pd.DateOffset(months=cm) 
             for cm in resp.cmbirth]
    resp['decade'] = (pd.DatetimeIndex(dates).year - 1900) // 10
    
    resp.cmmarrhx.replace([9997, 9998, 9999], np.nan, inplace=True)
    resp['agemarry'] = (resp.cmmarrhx - resp.cmbirth) / 12.0
    resp['age'] = (resp.cmintvw - resp.cmbirth) / 12.0

In [2]:
# //*****************************************
# //*** Build a probability mass function
# //*****************************************
# //*** Returns Series as a PMF
# //*****************************************
def build_pmf(input_series):
    output_series = input_series.copy()
    total_values = input_series.sum()
    for value,freq in output_series.items():
        #print(f"{value} {freq} {total_values} {freq/total_values}")
        output_series.loc[value] = freq/total_values
    return output_series

# //*** Build a Cumulative Distribution Function from a Probability Mass Function
# //*** Returns a Series
def build_cdf(input_series):
    # //*** If input is not panda or pd series, try to convert it
    if not isinstance(input_series,pd.core.series.Series):
        input_series = pd.Series(input_series)
        
    # //*** If input is np.Array
    output_series = input_series.copy()
    cumulative_value = 0
    for value,freq in output_series.items():
        #print(f"{value} {freq} {cumulative_value} {freq + cumulative_value}")
        cumulative_value = freq + cumulative_value
        output_series.loc[value] = cumulative_value
    return output_series

In [3]:
# //****************************************************************************************
# //*** Set Working Directory to thinkstats folder.
# //*** This pseudo-relative path call should work on all Stoneburner localized projects. 
# //****************************************************************************************

import os
import sys
workingPath = os.getcwd().replace("coding", "ThinkStats2\\code")
sys.path.insert(1, workingPath)
os.chdir(workingPath)

In [4]:
# //*** Imports and Load Data
import nsfg
import thinkstats2
import thinkplot
import first
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

resp6 = nsfg.ReadFemResp(dct_file='2002FemResp.dct',dat_file='2002FemResp.dat.gz')
resp7 = nsfg.ReadFemResp(dct_file='2006_2010_FemRespSetup.dct',dat_file='2006_2010_FemResp.dat.gz')

CleanData(resp6)

CleanData(resp7)

#//*** Combine 2002 and 2010 datasets
resp = pd.concat([resp6,resp7], sort=False)

**Exercise:**    In NSFG Cycles 6 and 7, the variable `cmdivorcx` contains the date of divorce for the respondent’s first marriage, if applicable, encoded in century-months.

Compute the duration of marriages that have ended in divorce, and the duration, so far, of marriages that are ongoing. Estimate the hazard and survival curve for the duration of marriage.

Use resampling to take into account sampling weights, and plot data from several resamples to visualize sampling error.

Consider dividing the respondents into groups by decade of birth, and possibly by age at first marriage.

In [96]:

#resp['notdivorced'] = resp.cmdivorcx.isnull().astype(int)
#resp['duration'] = (resp.cmdivorcx - resp.cmmarrhx) / 12.0
#resp['durationsofar'] = (resp.cmintvw - resp.cmmarrhx) / 12.0

#//*** There are a number of coding errors with durationsofar. A few marriages and divorces are listed 
#//*** are negative lengths. Drop these rows
resp.drop(index=resp[ resp['durationsofar'] <= 0 ].index, inplace=True)
resp.drop(index=resp[ resp['duration'] <= 0 ].index, inplace=True)



#//*** Not divorced attribute generate NaN for duration of marriage. 
#//*** Replace the NaN with durationsofar. This generates a length for all marriages

resp['duration'].fillna(resp['durationsofar'], inplace=True)

#//*** Generate histogram of durations. Rounded Continuous values to tenths for better value_counting / binning
duration_hist = round(resp[resp.evrmarry==1]['duration'],1).value_counts().sort_index()

#//*** Build CDF from PMF
duration_cdf = build_cdf( build_pmf(duration_hist) )

#//*** Build the Compimentary Survival Values
duration_survival = 1 - duration_cdf

#for key,value in duration_survival.items():
#    print(f"{key} : {value}")
    
hazard_all_subjects = build_hazard_function(duration_survival)

print(hazard_all_subjects)




{0.9958115183246073: 0.005906155548582204, 0.9857591623036649: -0.00967317251681521, 0.9812565445026178: -0.012874284651154477, 0.9754973821989529: -0.018920823039346835, 0.9700523560209424: -0.02455040736420433, 0.9648167539267015: -0.030842026190511462, 0.9606282722513089: -0.02847135713609028, 0.9501570680628272: -0.04334083804252886, 0.943979057591623: -0.04914351589700694, 0.9374869109947643: -0.05480617512162134, 0.930261780104712: -0.05623079080929061, 0.917696335078534: -0.07659850745912011, 0.9124607329842932: -0.07973573247359056, 0.9053403141361256: -0.08922364607668964, 0.9004188481675393: -0.09225471852626232, 0.8938219895287958: -0.09950040972143848, 0.8878534031413612: -0.09905543044750553, 0.8762303664921466: -0.11552393561110386, 0.8690052356020942: -0.12087306298809752, 0.8602094240837697: -0.1301740208340635, 0.851937172774869: -0.1319615489557111, 0.838219895287958: -0.15253588235101767, 0.8304712041884816: -0.16196354552782144, 0.8241884816753926: -0.16907793936386

In [108]:
#//*** Build hazard function from a cdf
#//*** Hazard Function: (1-CDF(x)) - (1-CDF(x+1)) / (1-CDF(x))
#//*** Difference between a value and the next value (ie the difference in two values) divided by the first value
#//*** This is a percentage of difference between sequential survivor function (1-CDF[x]) values
#//**** Returns a dictionary of hazard values
def build_hazard_function(sf):
    #//*** Convert cdf to survival function
    #sf = 1 - pd.Series(input_cdf)
    
    #//*** output dictionary
    out_dict= {}
    
    for index,value in enumerate(sf):
        #//*** Skip the last value since it generates an out of array error.
        #//*** Had troubles parsing a series len(-1) so did it this way
        if index < len(sf)-1:
            if sf.iloc[index] != 0:
                #//*** Value = Hazard Value
                out_dict[value] = ( sf.iloc[index] - sf.iloc[index+1] / sf.iloc[index] )
            else:
                out_dict[value] = 0
    return out_dict


In [111]:
#//*** Estimate the hazard function by Kaplan-Meier
#//*** Generates Risk for each specific Value. 
#//**** Round Ages 
complete = round(resp[resp.evrmarry==1]['agemarry'].dropna(),1)
ongoing = round(resp[resp.evrmarry==0]['age'],1)

hist_complete_counter = Counter(complete)
hist_ongoing_counter = Counter(ongoing)
#print(hist_complete)
#ts = list(hist_complete | hist_ongoing)
#ts.sort()
#print(ts)


#//*** Build Histogram Dictionary of Complete and ongoing
#//*** No reason to import a Library for Basic stuff
#//*** Usual histogram method of value_counts and sort_index, combined with a for items loop
#//*** to convert to a dictionary
hist_combined_list = []    

hist_complete = {}
for index, value in complete.value_counts().sort_index().items():
    #//*** Add Unique value to the combined list
    if index not in hist_complete.keys():
        hist_combined_list.append(index)
    
    #//*** Add to Dictionary
    hist_complete[index] = value

hist_ongoing = {}
for index, value in ongoing.value_counts().sort_index().items():
    #//*** Add Unique value to the combined list
    if index not in hist_complete.keys():
        hist_combined_list.append(index)

    #//*** Add to Dictionary
    hist_ongoing[index] = value

at_risk = len(complete) + len(ongoing)

survival_curve = pd.Series(index=hist_combined_list,dtype='float')

#print(hist_complete)

#//*** Go through each value in the ongoing and complete dictionaries
#//*** Get the count of each, or zero if not found
for x in hist_combined_list:
    
    ended = hist_complete_counter[x]
    censored = hist_ongoing_counter[x]
    #if x in hist_complete.keys():
    #    ended = hist_complete[x]
    #else:
    #    ended = 0
        
    #if x in hist_ongoing.keys():
    #    censored = hist_ongoing[x]
    #else:
    #    censored = 0
    
    #//*** Calculate the percentage of ended vs the remaining at risk
    #print(f"{x} {ended/at_risk}")
    survival_curve[x] = ended / at_risk
    
    #print(f"{ended} : {censored} : { lams[t] }")
    
    #//*** Reduced the at_risk total by the totals found
    at_risk -= ended + censored


kaplan_meier_hazard_function = build_hazard_function(survival_curve)

print(kaplan_meier_hazard_function)


{5.056378621631188e-05: -1.0000000025568259, 5.05663430420712e-05: -1.0000000025570843, 5.056890012642225e-05: -1.000000002557343, 5.057145746940427e-05: -1.0000000025576015, 5.057401507105649e-05: -1.0000000025578604, 5.057657293141817e-05: -1.000000002558119, 5.0579131050528554e-05: -1.0000000025583777, 5.058168942842691e-05: -1.0000000025586369, 5.0584248065152515e-05: -1.0000000025588955, 5.058680696074464e-05: -1.0000000025591544, 5.0589366115242574e-05: -1.0000000025594133, 5.0591925528685624e-05: -2.0000505970448734, 0.00010118897040222615: -4.000303607872183, 0.00040479684258462785: -0.24969644334977173, 0.00010124019235636548: -2.50015188591507, 0.00025312610742672: -0.5998987880104587, 0.00015191411788535549: -2.0001519602806965, 0.00030387439858191946: -0.4998481089845682, 0.00015198338315010893: -3.3336880382355494, 0.0005066882853668423: -0.3996960897740472, 0.0002027780594139714: -2.2502535651106497, 0.00045634317006388805: -0.8896963260015958, 0.0004062150908906266: -1.7

In [7]:

"""Estimates the hazard function by Kaplan-Meier.

http://en.wikipedia.org/wiki/Kaplan%E2%80%93Meier_estimator

complete: list of complete lifetimes
ongoing: list of ongoing lifetimes
label: string
verbose: whether to display intermediate results
"""

hist_complete = Counter(complete)
hist_ongoing = Counter(ongoing)

ts = list(hist_complete | hist_ongoing)
ts.sort()

at_risk = len(complete) + len(ongoing)

lams = pd.Series(index=ts)
for t in ts:
    ended = hist_complete[t]
    censored = hist_ongoing[t]

    lams[t] = ended / at_risk
    if verbose:
        print(t, at_risk, ended, censored, lams[t])
    at_risk -= ended + censored

#    return survival.HazardFunction(lams, label=label)

<ipython-input-7-2484795dc41b>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)


NameError: name 'verbose' is not defined

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE

- ## Chapter X, Exercise X



In [ ]:
# //*** CODE HERE